In [ ]:
# Install required packages quietly
!pip install -q openai anthropic mistralai

In [ ]:
# Import necessary modules
import os
from utils import common
from experiment_resources.questionnaires import BFI44 # You can replace this with another questionnaire
import experiment_functions.questionnaire as questionnaire
import experiment_functions.qa_text_generation as qa_text_generation
import utils.qa_classifier as qa_classifier

In [ ]:
# Set API keys for the models you intend to use in the experiments
os.environ['OPENAI_API_KEY'] = 'your_openai_api_key'
os.environ['ANTHROPIC_API_KEY'] = 'your_anthropic_api_key'
os.environ['MISTRAL_API_KEY'] = 'your_mistral_api_key'

In [ ]:
# Initialize common settings
common.initialize()

# Define experiment settings

In [ ]:
# Questionnaire experiment settings
questionnaire_settings = {
    'models': {
        'gpt-3.5-turbo': {'temperature': 0.9},
        'gpt-4o': {'temperature': 0.9},
        'claude-3-haiku-20240307': {'temperature': 0.9},
        'open-mixtral-8x22b': {'temperature': 0.9}
        # Add more models here if needed or change temperature
    },
    'questionnaire_module': BFI44  # You can replace this with another questionnaire
}

# Text generation experiment settings
text_generation_settings = {
    'models': {
        'gpt-3.5-turbo': {'temperatures': [0.7]},
        'gpt-4o': {'temperatures': [0.7]},
        'claude-3-haiku-20240307': {'temperatures': [0.7]},
        'open-mixtral-8x22b': {'temperatures': [0.7]}
        # Add more models and temperatures here if needed
    },
    'questions_file': 'questions.csv'
}

# Visualization settings
visualization_settings = {
    'trait_name_mapping': {  # Mapping of trait names
        'Agreeableness': 'Agreeableness',
        'Conscientiousness': 'Conscientiousness',
        'Extraversion': 'Extraversion',
        'Neuroticism': 'Neuroticism',
        'Openness': 'Openness to experience'
    },
    'model_name_mapping': {  # Mapping of model names
        'gpt-3.5-turbo': 'GPT-3.5 Turbo',
        'gpt-4o': 'GPT-4o',
        'claude-3-haiku-20240307': 'Claude-3-haiku',
        'open-mixtral-8x22b': 'Mixtral-8x22b'
        # Add other models if necessary
    }
}

# Classifier settings
text_analysis_settings = {
    'model_name': 'gpt-4o',
    'temperature': 0
}

# Step 1. Questionnaire

In [ ]:
# Run the process of LLMs questionnaire answering
questionnaire.run_experiment(questionnaire_settings)

In [ ]:
# Visualize results
questionnaire.visualize_results(
    df_answers='last', # Use 'last' for the last experiment or specify an experiment folder name (e.g, 'questionnaire_20241111_180643')
    visualization_settings=visualization_settings,
    visualization_type='histogram'
)

# Step 2. Text generation

In [ ]:
# Run the process of text generation experiment
qa_text_generation.run_experiment(text_generation_settings)

In [ ]:
# Run the process of personality detection by LLM-based classifier
qa_classifier.run_analysis(
    df_answers='last', # Use 'last' for the last experiment or specify an experiment folder name (e.g, 'qa_text_generation_20241111_182218')
    settings=text_analysis_settings
)

In [ ]:
# Visualize the results
qa_text_generation.visualize_results(
    df_answers='last', # Use 'last' for the last experiment or specify an experiment folder name (e.g, 'qa_text_generation_20241111_182218')
    visualization_settings=visualization_settings,
    visualization_type='confusion matrices' # Options: 'confusion matrices', 'similarity'
)